This is the code that goes into the hangpoint device with the 
camera and orientation sensor, connecting to the Blackview phone 
through port 9019.  (Ported over from AirTemperatureTech)

In [1]:
%serialconnect

Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [5]:
%sendtofile --quiet --source BNO055_funcs.py
%sendtofile --quiet --source BlueFly_funcs.py
%sendtofile --quiet --source NewDataLogger_funcs.py
%sendtofile --quiet --source UDPblackviewphone.py


Sent 134 lines (4341 bytes) to BNO055_funcs.py.
Sent 112 lines (4469 bytes) to BlueFly_funcs.py.
Sent 92 lines (2739 bytes) to NewDataLogger_funcs.py.
Sent 41 lines (1067 bytes) to UDPblackviewphone.py.


In [3]:
%sendtofile deviceletter.txt
@

Sent 1 lines (1 bytes) to deviceletter.txt.


In [4]:
%sendtofile main.py

from machine import Pin, UART
import os, time, math, ustruct, urandom
from NewDataLogger_funcs import ConnectBluefly, readlogbluefly
from BNO055_funcs import InitBNO055, readhexlifyBNO055

# uartBlueFly = UART(1, baudrate=57600, rx=13, tx=17)
uartBNO055 = UART(2, baudrate=115200, rx=2, tx=18)

pled = Pin(14, Pin.OUT)
for i in range(10):
    time.sleep_ms(200)
    pled.value(i%2)

pled.value(1)
for i in range(5):
    try:
        k = InitBNO055(uartBNO055)
        print(k)
        break
    except OSError:
        print("Bad BNO055", i)
    time.sleep(0.05)
    pled.value(0)
    time.sleep(0.1)
    pled.value(1)
    time.sleep(0.5)
pled.value(0)

ConnectBluefly(None, vol=20)

for i in range(11):
    pled.value(i%2)
    time.sleep_ms(200)

deviceletter = open("deviceletter.txt", "r").read()
print("deviceletter", deviceletter)


from UDPblackviewphone import connectActivePhone
while not connectActivePhone(pled):
    for j in range(3):
        for i in range(7):
            pled.value(i%2)
            time.sleep_ms(100)
        time.sleep_ms(800)

        
import socket
androidipnumber = "192.168.43.1"
portnumber = 9042
        
ubs = bytearray("Ut00000000i00\n")
mubs = memoryview(ubs)

prevflushstamp = 0
nextledonstamp = 0
gpsfixes = 0

while True:
    try:
        ss = socket.socket()
        ss.settimeout(1)
        print(ss)
        ss.connect(socket.getaddrinfo(androidipnumber, portnumber)[0][-1])
        s = ss.makefile('rwb', 0)
        print(s.readline())
        s.write(b"%c%c%c%c"%(deviceletter,deviceletter,deviceletter,deviceletter))
        dwrite = s.write
        while True:
            tstamp = time.ticks_ms()
            if tstamp > nextledonstamp:
                pv = (0 if pled.value() else 1)
                pled.value(pv)
                mubs[2:10] = b"%08X" % tstamp
                mubs[12] = 48+pv #'0'or'1'
                dwrite(ubs)
                nextledonstamp = tstamp + urandom.randint(5000, 10000)//(10 if pv else 1)

            lbf = readlogbluefly(dwrite)
            bno055rec = readhexlifyBNO055(10)
            if bno055rec:
                dwrite(bno055rec)
                
    except OSError as e:
        print("OSError", e)




        
        
        

Sent 92 lines (2215 bytes) to main.py.


In [6]:
%disconnect

attempt to exit paste mode
[\r\x03\x02] b'\r\nMicroPython v1.9.4-651-g0f6f86ca4 on 2018-10-17; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> '
Closing serial Serial<id=0x7f21bd0b0e48, open=True>(port='/dev/ttyUSB0', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)


In [23]:
import os
help(os.stat("calibfile.txt"))


Traceback (most recent call last):
  File "<stdin>", line 2, in <module>
OSError: [Errno 2] ENOENT


In [10]:
%sendtofile hotspots.txt

BV6000         beckaaaa    192.168.43.1   9042
JulianS9       beckaaaa    192.168.43.1   9042
DoESLiverpool  decafbad00  node-red.dynamicdevices.co.uk   4006

Sent 3 lines (157 bytes) to hotspots.txt.


In [12]:
%serialconnect

Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [20]:
%ls

Listing directory '/'.
     3165    BME280_funcs.py
     4341    BNO055_funcs.py
     2732    BNO055serial_funcs.py
     4469    BlueFly_funcs.py
    14410    Dmain.py
      854    MLX90614_funcs.py
     5919    MLX90621_funcs.py
     1375    MS5611_funcs.py
     2739    NewDataLogger_funcs.py
     3820    OLED_driver.py
     4083    OLED_grapher.py
     1800    SDOF_funcs.py
     1090    SHT31D_funcs.py
     2040    SI7021_funcs.py
     1757    TSL561_funcs.py
     1067    UDPblackviewphone.py
      975    VL53L0X_funcs.py
     3190    VL6180_funcs.py
     8811    aswitch.py
    14738    asyn.py
      139    boot.py
      113    config.txt
     3180    device_detection.py
        1    deviceletter.txt
     2314    fmqttqueue.py
      157    hotspots.txt
             lib/
     1598    main.py
    25011    mqtt_as.py
     2004    ubxminimal.py


In [18]:
uart2.read()
uart2.write(b"\xAA\x01\x00\x06")  # request chip_id and firmware version
time.sleep_ms(20)
r = uart2.read()
if r is not None and len(r) == 8 and r[:3] == b'\xbb\x06\xa0':
    swversion = "%d.%d" % (r[6], r[7])  # 8.3
    print("BNO055 orient")
time.sleep(0.2)


BNO055 orient


In [11]:
%sendtofile main.py

# charuco board bno055 only
from machine import Pin, UART
import os, time, math, ustruct, urandom
from BNO055_funcs import InitBNO055, readhexlifyBNO055

pled = Pin(2, Pin.OUT)
for i in range(10):
    time.sleep_ms(200)
    pled.value(i%2)

for i in range(5):
    try:
        k = InitBNO055(tx=17, rx=16)
        print(k)
        break
    except OSError:
        print("Bad BNO055", i)
    time.sleep(0.05)
    pled.value(0)
    time.sleep(0.1)
    pled.value(1)
    time.sleep(0.5)
pled.value(0)

for i in range(11):
    pled.value(i%2)
    time.sleep_ms(200)

deviceletter = open("deviceletter.txt", "r").read()
print("deviceletter", deviceletter)

from UDPblackviewphone import connectActivePhone
while not connectActivePhone(pled):
    for j in range(3):
        for i in range(7):
            pled.value(i%2)
            time.sleep_ms(100)
        time.sleep_ms(800)

        
import socket
androidipnumber = "192.168.43.1"
portnumber = 9042
        
while True:
    try:
        ss = socket.socket()
        ss.settimeout(1)
        print(ss)
        ss.connect(socket.getaddrinfo(androidipnumber, portnumber)[0][-1])
        s = ss.makefile('rwb', 0)
        print(s.readline())
        s.write(b"%c%c%c%c"%(deviceletter,deviceletter,deviceletter,deviceletter))
        dwrite = s.write
        while True:
            bno055rec = readhexlifyBNO055(10)
            if bno055rec:
                pled.value(1)
                dwrite(bno055rec)
                pled.value(0)
            time.sleep_ms(5)
                
    except OSError as e:
        print("OSError", e)
        
        

Sent 66 lines (1598 bytes) to main.py.


In [40]:
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [48]:
# Bike rolling single button logger

%sendtofile main.py

from machine import Pin, UART
import os, time, math, ustruct, urandom

pled = Pin(2, Pin.OUT)
p0 = Pin(0, Pin.IN)  # boot button

for i in range(10):
    time.sleep_ms(200)
    pled.value(i%2)

deviceletter = "@"

from UDPblackviewphone import connectActivePhone
while not connectActivePhone(pled):
    for j in range(3):
        for i in range(7):
            pled.value(i%2)
            time.sleep_ms(100)
        time.sleep_ms(800)

import socket
androidipnumber = "192.168.43.1"
portnumber = 9042
        
ubs = bytearray("Ut00000000i00\n")
mubs = memoryview(ubs)

prevflushstamp = 0
nextledonstamp = 0
gpsfixes = 0

while True:
    try:
        ss = socket.socket()
        ss.settimeout(1)
        print(ss)
        ss.connect(socket.getaddrinfo(androidipnumber, portnumber)[0][-1])
        s = ss.makefile('rwb', 0)
        print(s.readline())
        s.write(b"%c%c%c%c"%(deviceletter,deviceletter,deviceletter,deviceletter))
        dwrite = s.write
        while True:
            tstamp = time.ticks_ms()
            if tstamp > nextledonstamp:
                pv = p0.value()
                pled.value(pv)
                mubs[2:10] = b"%08X" % tstamp
                mubs[12] = 48+pv #'0'or'1'
                s.write(ubs)
                nextledonstamp = tstamp + 100
                
    except OSError as e:
        print("OSError", e)



Sent 54 lines (1354 bytes) to main.py.
